In [1]:
import sys,os
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0,project_root)

from src.data_handling.load_data import *
from src.data_handling.preprocessing import *
from src.models.model_setup import *
from src.trainer.training import *
from src.trainer.file_utils import *

c:\Users\Hector Auvinen\Documents\GitHub\adapter_experiments\adapter_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# supports tasks:
# sst2
# cb
# rte
# sick
# mrpc
# boolq
task = "boolq"
model_name = "bert-base-uncased"
# num_labels = 2
output_dir = "C:/Users/Hector Auvinen/Desktop/eval_results"
#logger = logging.getLogger(__name__)
#logger.setLevel(logging.INFO)

In [3]:
data = load_hf_dataset(task,debug=False)

tokenizer = get_tokenizer(model_name)

encode = get_encoding(task)

using bert tokenizer
getting encoding:
<function encode_rte at 0x000001E3A5592280>


In [4]:
#def preprocess_dataset(dataset,encoding_func,tokenizer):
dataset = preprocess_dataset(data,encode,tokenizer)

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returne

mapped
{'train': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']}


In [6]:
#id2label = {id: label for (id,label) in enumerate(dataset["train"].features["labels"].names)}
#num_labels = len(id2label)
num_labels = get_label_count(dataset)
print("labels",num_labels)
model = setup_model(model_name,num_labels,dataset)

adapter_config = adapters.BnConfig(
                          output_adapter=True,
                          mh_adapter=False,
                          reduction_factor=2,
                          non_linearity="relu")

model = add_clf_adapter(task_name=task,model=model,num_labels=num_labels,adapter_config=adapter_config)

default_args = TrainingParameters()
default_args.lr_scheduler_type = "linear"


labels 2


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
train_args = get_training_arguments(default_args)

trainer = get_trainer(train_args,dataset,model,early_stopping=3)

trainer.train()

                                                  
  1%|          | 50/9360 [00:24<53:20,  2.91it/s]

{'eval_loss': 0.7221770882606506, 'eval_accuracy': 0.5270758122743683, 'eval_runtime': 6.1002, 'eval_samples_per_second': 45.408, 'eval_steps_per_second': 5.737, 'epoch': 0.16}


                                                   
  1%|          | 100/9360 [00:47<53:01,  2.91it/s]

{'eval_loss': 0.6917835474014282, 'eval_accuracy': 0.5306859205776173, 'eval_runtime': 6.0676, 'eval_samples_per_second': 45.652, 'eval_steps_per_second': 5.768, 'epoch': 0.32}


                                                    
  2%|▏         | 150/9360 [01:10<51:47,  2.96it/s]

...

                                                    
  7%|▋         | 650/9360 [04:52<47:05,  3.08it/s]

{'eval_loss': 0.7050133347511292, 'eval_accuracy': 0.5992779783393501, 'eval_runtime': 5.5648, 'eval_samples_per_second': 49.777, 'eval_steps_per_second': 6.29, 'epoch': 2.08}


                                                    
  7%|▋         | 700/9360 [05:14<1:04:46,  2.23it/s]

{'eval_loss': 0.6724349856376648, 'eval_accuracy': 0.6064981949458483, 'eval_runtime': 5.7438, 'eval_samples_per_second': 48.226, 'eval_steps_per_second': 6.094, 'epoch': 2.24}
{'train_runtime': 314.1879, 'train_samples_per_second': 237.756, 'train_steps_per_second': 29.791, 'train_loss': 0.6812251935686384, 'epoch': 2.24}


TrainOutput(global_step=700, training_loss=0.6812251935686384, metrics={'train_runtime': 314.1879, 'train_samples_per_second': 237.756, 'train_steps_per_second': 29.791, 'train_loss': 0.6812251935686384, 'epoch': 2.24})

In [ ]:
eval_results = trainer.evaluate()

100%|██████████| 7/7 [00:01<00:00,  6.88it/s]


In [ ]:
write_eval_results(eval_results,output_dir,task,trainer,adapter_config)

Writing eval results
{'eval_loss': 0.691024899482727, 'eval_accuracy': 0.8392857142857143, 'eval_runtime': 1.2199, 'eval_samples_per_second': 45.905, 'eval_steps_per_second': 5.738, 'epoch': 30.0}


TypeError: write() argument must be str, not dict

In [ ]:
"""for attr_name, attr_value in vars(default_args).items():
    print(f"{attr_name}: {attr_value}")"""
"""
    train_args = {"label_names":["labels"],
    "evaluation_strategy":"steps",
    "learning_rate":1e-4,
    "num_train_epochs":1,
    "per_device_train_batch_size":8,
    "per_device_eval_batch_size":8,
    "eval_steps":50,
    "logging_steps":200,
    "output_dir":"/eval_results",
    "overwrite_output_dir":True,
    "remove_unused_columns":False,
    "lr_scheduler_type":'linear',
    "load_best_model_at_end":True,
    "metric_for_best_model" : "accuracy",
    "early_stopping_patience":3,
    "save_total_limit":5
}
"""

'\n    train_args = {"label_names":["labels"],\n    "evaluation_strategy":"steps",\n    "learning_rate":1e-4,\n    "num_train_epochs":1,\n    "per_device_train_batch_size":8,\n    "per_device_eval_batch_size":8,\n    "eval_steps":50,\n    "logging_steps":200,\n    "output_dir":"/eval_results",\n    "overwrite_output_dir":True,\n    "remove_unused_columns":False,\n    "lr_scheduler_type":\'linear\',\n    "load_best_model_at_end":True,\n    "metric_for_best_model" : "accuracy",\n    "early_stopping_patience":3,\n    "save_total_limit":5\n}\n'